# Pretrained model only

### Load data

In [1]:
import pickle

train_filename = "C:/Users/behl/Desktop/minor/train_data_sample_rgb.p"
(train_labels, train_data, train_tensors) = pickle.load(open(train_filename, mode='rb'))

valid_filename = "C:/Users/behl/Desktop/minor/valid_data_sample_rgb.p"
(valid_labels, valid_data, valid_tensors) = pickle.load(open(valid_filename, mode='rb'))

test_filename = "C:/Users/behl/Desktop/minor/test_data_sample_rgb.p"
(test_labels, test_data, test_tensors) = pickle.load(open(test_filename, mode='rb'))

### CNN model

In [13]:
import time

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
# VGG16
# resnet50.ResNet50
# inception_v3.InceptionV3 299x299
# inception_resnet_v2.InceptionResNetV2 299x299

base_model = VGG16(weights='imagenet', include_top=False, input_shape=train_tensors.shape[1:])

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.2))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(50, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.summary()
add_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [14]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [15]:
import keras.backend as K

model.compile(optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), 
              loss='binary_crossentropy', 
              metrics=[binary_accuracy,
                       precision_threshold(threshold = 0.4), 
                       recall_threshold(threshold = 0.4), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.4),
                      precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      precision_threshold(threshold = 0.6), 
                       recall_threshold(threshold = 0.6), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.6)])

In [16]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
log = CSVLogger('C:/Users/behl/Desktop/minor/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='C:/Users/behl/Desktop/minor/pretrainedVGG.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

# model.fit(train_tensors, train_labels, 
#           validation_data=(valid_tensors, valid_labels),
#           epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

def train_generator(x, y, batch_size):
        train_datagen = ImageDataGenerator(
                        featurewise_center=False,  # set input mean to 0 over the dataset
                        samplewise_center=False,  # set each sample mean to 0
                        featurewise_std_normalization=False,  # divide inputs by std of the dataset
                        samplewise_std_normalization=False,  # divide each input by its std
                        zca_whitening=False,  # apply ZCA whitening
                        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                        horizontal_flip=True,  # randomly flip images
                        vertical_flip=False)  # randomly flip images
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield [x_batch, y_batch]

# Training with data augmentation. If shift_fraction=0., also no augmentation.
model.fit_generator(generator=train_generator(train_tensors, train_labels, batch_size),
                    steps_per_epoch=int(train_labels.shape[0] / batch_size),
                    validation_data=(valid_tensors, valid_labels),
                    epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Epoch 1/20


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Attempting to use uninitialized value training_1/SGD/Variable_28
	 [[{{node training_1/SGD/Variable_28/read}}]]
	 [[metrics_1/recall_1/Mean/_49]]
  (1) Failed precondition: Attempting to use uninitialized value training_1/SGD/Variable_28
	 [[{{node training_1/SGD/Variable_28/read}}]]
0 successful operations.
0 derived errors ignored.

### Metric

In [15]:
model.load_weights('saved_models/pretrainedVGG.best.from_scratch.hdf5')
prediction = model.predict(test_tensors)

In [16]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.644550 %
Recall: 0.555102 %
Fscore: 0.624426 %


In [17]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                       K.variable(value=prediction)))

0.66696835

In [18]:
prediction[:30]

array([[ 0.48835951],
       [ 0.26106429],
       [ 0.41754654],
       [ 0.64929706],
       [ 0.32506767],
       [ 0.60315681],
       [ 0.25606936],
       [ 0.56809974],
       [ 0.21197747],
       [ 0.29700339],
       [ 0.45825043],
       [ 0.699117  ],
       [ 0.19021288],
       [ 0.15274318],
       [ 0.57559574],
       [ 0.66964573],
       [ 0.4788951 ],
       [ 0.63720137],
       [ 0.57080215],
       [ 0.19700085],
       [ 0.49120948],
       [ 0.79610401],
       [ 0.2416992 ],
       [ 0.60802132],
       [ 0.56850374],
       [ 0.55580062],
       [ 0.72331935],
       [ 0.37036556],
       [ 0.64609927],
       [ 0.49083158]], dtype=float32)

# Extra data

In [1]:
import pickle

train_filename = "data_preprocessed/train_data_sample_rgb.p"
(train_labels, train_data, train_tensors) = pickle.load(open(train_filename, mode='rb'))

valid_filename = "data_preprocessed/valid_data_sample_rgb.p"
(valid_labels, valid_data, valid_tensors) = pickle.load(open(valid_filename, mode='rb'))

test_filename = "data_preprocessed/test_data_sample_rgb.p"
(test_labels, test_data, test_tensors) = pickle.load(open(test_filename, mode='rb'))

In [20]:
import time

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.layers import Input, merge, concatenate

base_model = applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=train_tensors.shape[1:])

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))

added_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


inp = Input(batch_shape=(None, train_data.shape[1]))
# out = Dense(8)(inp)
extra_model = Model(input=inp, output=inp)

x = concatenate([added_model.output,
           extra_model.output])

# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model =  Model(input=[added_model.input,
                extra_model.input],
                output=x)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 64, 64)   1792        input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 64, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 32, 32, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("in..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [21]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [22]:
model.compile(optimizer='sgd', loss='binary_crossentropy', 
              metrics=[binary_accuracy,
                       precision_threshold(threshold = 0.4), 
                       recall_threshold(threshold = 0.4), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.4),
                      precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      precision_threshold(threshold = 0.6), 
                       recall_threshold(threshold = 0.6), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.6)])

In [30]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
log = CSVLogger('saved_models/log_pretrained_extradata_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrained_extradata_CNN.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

model.fit([train_tensors,  train_data], train_labels, 
          validation_data=([valid_tensors, valid_data], valid_labels),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

# def train_generator(x1, x2, y, batch_size):
#         train_datagen = ImageDataGenerator(
#                         featurewise_center=False,  # set input mean to 0 over the dataset
#                         samplewise_center=False,  # set each sample mean to 0
#                         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#                         samplewise_std_normalization=False,  # divide each input by its std
#                         zca_whitening=False,  # apply ZCA whitening
#                         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#                         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#                         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#                         horizontal_flip=True,  # randomly flip images
#                         vertical_flip=False)  # randomly flip images
#         generator = train_datagen.flow((x1, x2), y, batch_size=batch_size)
#         while 1:
#             (x1_batch, x2_batch), y_batch = generator.next()
#             yield [[x1_batch, x2_batch], y_batch]

# # Training with data augmentation. If shift_fraction=0., also no augmentation.
# model.fit_generator(generator=train_generator(train_tensors, train_data, train_labels, batch_size),
#                     steps_per_epoch=int(train_labels.shape[0] / batch_size),
#                     validation_data=([valid_tensors, valid_data], valid_labels),
#                     epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Train on 3400 samples, validate on 1100 samples
Epoch 1/20
3400/3400 [==============================] - 16s 5ms/step - loss: 0.7016 - binary_accuracy: 0.5453 - precision_1: 0.4785 - recall_1: 0.7321 - fbeta_score_1: 0.5001 - precision_2: 0.4903 - recall_2: 0.3660 - fbeta_score_2: 0.3970 - precision_3: 0.3337 - recall_3: 0.1315 - fbeta_score_3: 0.2098 - val_loss: 0.7293 - val_binary_accuracy: 0.4709 - val_precision_1: 0.4618 - val_recall_1: 1.0000 - val_fbeta_score_1: 0.5153 - val_precision_2: 0.4650 - val_recall_2: 0.9792 - val_fbeta_score_2: 0.5174 - val_precision_3: 0.5316 - val_recall_3: 0.8313 - val_fbeta_score_3: 0.5692
Epoch 2/20
3400/3400 [==============================] - 16s 5ms/step - loss: 0.6728 - binary_accuracy: 0.5953 - precision_1: 0.5332 - recall_1: 0.7590 - fbeta_score_1: 0.5502 - precision_2: 0.5897 - recall_2: 0.4599 - fbeta_score_2: 0.5141 - precision_3: 0.5295 - recall_3: 0.2147 - fbeta_score_3: 0.3504 - val_loss: 0.6439 - val_binary_accuracy: 0.6436 - val_precisi

In [31]:
model.load_weights('saved_models/pretrained_extradata_CNN.best.from_scratch.hdf5')
prediction = model.predict([test_tensors, test_data])

In [32]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.647191 %
Recall: 0.587755 %
Fscore: 0.634361 %


In [33]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                       K.variable(value=prediction)))

0.67511314

In [34]:
prediction[:30]

array([[ 0.43854904],
       [ 0.28271237],
       [ 0.32896599],
       [ 0.76249766],
       [ 0.31618291],
       [ 0.77488339],
       [ 0.3089357 ],
       [ 0.72916156],
       [ 0.21439876],
       [ 0.38674796],
       [ 0.4488712 ],
       [ 0.7688958 ],
       [ 0.28179762],
       [ 0.23615587],
       [ 0.70831102],
       [ 0.72348529],
       [ 0.36050457],
       [ 0.72639018],
       [ 0.43309873],
       [ 0.25428677],
       [ 0.41418654],
       [ 0.80132085],
       [ 0.32700819],
       [ 0.79180586],
       [ 0.75714654],
       [ 0.73956227],
       [ 0.75789005],
       [ 0.25184935],
       [ 0.62190068],
       [ 0.51263815]], dtype=float32)

# Train with extra data and spacial transformer

In [1]:
import pickle

train_filename = "data_preprocessed/train_data_sample_rgb.p"
(train_labels, train_data, train_tensors) = pickle.load(open(train_filename, mode='rb'))

valid_filename = "data_preprocessed/valid_data_sample_rgb.p"
(valid_labels, valid_data, valid_tensors) = pickle.load(open(valid_filename, mode='rb'))

test_filename = "data_preprocessed/test_data_sample_rgb.p"
(test_labels, test_data, test_tensors) = pickle.load(open(test_filename, mode='rb'))

In [43]:
import time
import numpy as np

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.layers import Input, merge, concatenate
from spatial_transformer import SpatialTransformer

def locnet():
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((64, 6), dtype='float32')
    weights = [W, b.flatten()]
    locnet = Sequential()

    locnet.add(Conv2D(16, (7, 7), padding='valid', input_shape=train_tensors.shape[1:]))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(32, (5, 5), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(64, (3, 3), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))

    locnet.add(Flatten())
    locnet.add(Dense(128, activation='elu'))
    locnet.add(Dense(64, activation='elu'))
    locnet.add(Dense(6, weights=weights))

    return locnet

base_model = applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=train_tensors.shape[1:])

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))

added0_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

stn_model = Sequential()
stn_model.add(Lambda(
    lambda x: 2*x - 1.,
    input_shape=train_tensors.shape[1:],
    output_shape=train_tensors.shape[1:]))
stn_model.add(BatchNormalization())
stn_model.add(SpatialTransformer(localization_net=locnet(),
                                 output_size=train_tensors.shape[1:3]))

added_model = Model(inputs=stn_model.input, outputs=added0_model(stn_model.output))

inp = Input(batch_shape=(None, train_data.shape[1]))
# out = Dense(8)(inp)
extra_model = Model(input=inp, output=inp)

x = concatenate([added_model.output,
           extra_model.output])

# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model =  Model(input=[added_model.input,
                extra_model.input],
                output=x)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lambda_3_input (InputLayer)     (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 64, 64, 3)    0           lambda_3_input[0][0]             
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 64, 64, 3)    12          lambda_3[0][0]                   
__________________________________________________________________________________________________
spatial_transformer_3 (SpatialT (None, 64, 64, 3)    247270      batch_normalization_3[0][0]      
__________________________________________________________________________________________________
model_18 (

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("in..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:73: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [44]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [46]:
model.compile(optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), 
              loss='binary_crossentropy', 
              metrics=[binary_accuracy,
                       precision_threshold(threshold = 0.4), 
                       recall_threshold(threshold = 0.4), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.4),
                      precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      precision_threshold(threshold = 0.6), 
                       recall_threshold(threshold = 0.6), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.6)])

In [47]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
log = CSVLogger('saved_models/log_pretrained_extradata_stn_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/log_pretrained_extradata_stn_CNN.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

model.fit([train_tensors, train_data], train_labels, 
          validation_data=([valid_tensors, valid_data], valid_labels),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

# def train_generator(x, y, batch_size):
#         train_datagen = ImageDataGenerator(
#                         featurewise_center=False,  # set input mean to 0 over the dataset
#                         samplewise_center=False,  # set each sample mean to 0
#                         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#                         samplewise_std_normalization=False,  # divide each input by its std
#                         zca_whitening=False,  # apply ZCA whitening
#                         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#                         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#                         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#                         horizontal_flip=True,  # randomly flip images
#                         vertical_flip=False)  # randomly flip images
#         generator = train_datagen.flow(x, y, batch_size=batch_size)
#         while 1:
#             x_batch, y_batch = generator.next()
#             yield [x_batch, y_batch]

# # Training with data augmentation. If shift_fraction=0., also no augmentation.
# model.fit_generator(generator=train_generator(train_tensors, train_labels, batch_size),
#                     steps_per_epoch=int(train_labels.shape[0] / batch_size),
#                     validation_data=(valid_tensors, valid_labels),
#                     epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Train on 3400 samples, validate on 1100 samples
Epoch 1/20
3400/3400 [==============================] - 19s 6ms/step - loss: 0.7861 - binary_accuracy: 0.5291 - precision_1: 0.4770 - recall_1: 0.6241 - fbeta_score_1: 0.4949 - precision_2: 0.4906 - recall_2: 0.4579 - fbeta_score_2: 0.4735 - precision_3: 0.5074 - recall_3: 0.3053 - fbeta_score_3: 0.4293 - val_loss: 0.6623 - val_binary_accuracy: 0.5609 - val_precision_1: 0.5511 - val_recall_1: 0.8046 - val_fbeta_score_1: 0.5840 - val_precision_2: 0.5571 - val_recall_2: 0.2083 - val_fbeta_score_2: 0.3974 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_fbeta_score_3: 0.0000e+00
Epoch 2/20
3400/3400 [==============================] - 18s 5ms/step - loss: 0.7123 - binary_accuracy: 0.5591 - precision_1: 0.5053 - recall_1: 0.6611 - fbeta_score_1: 0.5262 - precision_2: 0.5235 - recall_2: 0.4474 - fbeta_score_2: 0.4978 - precision_3: 0.5553 - recall_3: 0.2656 - fbeta_score_3: 0.4390 - val_loss: 0.6454 - val_binary_accuracy: 0.6236 -

In [48]:
model.load_weights('saved_models/log_pretrained_extradata_stn_CNN.best.from_scratch.hdf5')
prediction = model.predict([test_tensors, test_data])

In [49]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.641791 %
Recall: 0.614286 %
Fscore: 0.636095 %


In [50]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                       K.variable(value=prediction)))

0.6769231